# Chemicals e Pubchem

In [1]:
from helper_knn import *

db = pd.read_csv('lc_db_processed.csv').drop(columns = 'Unnamed: 0')
db = pd.concat([db,
                pd.DataFrame(pd.DataFrame(db['pubchem2d'].values).\
                             apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                        axis = 1)[0].to_list(),
                   columns = ['pub'+ str(i) for i in range(1,882)])],
               axis = 1)

numerical = ['atom_number', 'bonds_number','Mol', 'MorganDensity', 'LogP',
            'alone_atom_number', 'doubleBond', 'tripleBond', 'ring_number', 'oh_count', 'MeltingPoint', 'WaterSolubility']

categorical = ['test_cas', 'fish', 'smiles', 'pubchem2d', 'obs_duration_mean',
               'conc1_type', 'exposure_type', 'control_type', 'media_type',
               'application_freq_unit', 'species', 'class', 'tax_order', 'family', 'genus']
db.drop(columns = categorical, inplace = True)

X = db.drop(columns = 'conc1_mean')
y = np.where(db['conc1_mean']> 1, 0,1)

# Logistic Regression -- fatta
sta nel file Regressione Logistica
Risultati: 

Accuracy: 0.8548787786221854 se: 0.0014016778744102632

Sensitivity: 0.7925727120942547 se: 0.006806487851100124

Specificity: 0.8937571771387468 se: 0.0037625931835931054

# K-NN

In [2]:
accs = []
sens = []
specs = []
precs = []

kf = KFold(n_splits=5, shuffle=True, random_state = 5645)

for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    minmax = MinMaxScaler()
    minmax.fit(X_train[numerical])

    new_train = X_train.copy()
    new_train.loc[:, numerical] = minmax.transform(X_train[numerical])

    new_test = X_test.copy()
    new_test.loc[:, numerical] = minmax.transform(X_test[numerical])
    
    
    knn = KNeighborsClassifier(n_neighbors = 1)
    knn.fit(new_train, y_train)
    y_pred = knn.predict(new_test)
    accs.append(accuracy_score(y_test, y_pred))
    sens.append(recall_score(y_test, y_pred))
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    specs.append(tn/(tn+fp))
    precs.append(precision_score(y_test, y_pred))
    
print('Accuracy:   ', np.mean(accs), 'se:', sem(accs))
print('Sensitivity:', np.mean(sens), 'se:', sem(sens))
print('Specificity:', np.mean(specs), 'se:', sem(specs))
print('Precision:  ', np.mean(precs), 'se:', sem(precs))

Accuracy:    0.8386822054163465 se: 0.003511490642735706
Sensitivity: 0.7750543592155392 se: 0.013999533749634328
Specificity: 0.8780690876469682 se: 0.005848042260615696
Precision:   0.7980879344718407 se: 0.007086587169302267


# Random Forest

In [3]:
from sklearn.ensemble import RandomForestClassifier

accs = []
sens = []
specs = []
precs = []

kf = KFold(n_splits=5, shuffle=True, random_state = 5645)

for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    minmax = MinMaxScaler()
    minmax.fit(X_train[numerical])

    new_train = X_train.copy()
    new_train.loc[:, numerical] = minmax.transform(X_train[numerical])

    new_test = X_test.copy()
    new_test.loc[:, numerical] = minmax.transform(X_test[numerical])
    
    
    rfc = RandomForestClassifier()
    rfc.fit(new_train, y_train)
    y_pred = rfc.predict(new_test)
    accs.append(accuracy_score(y_test, y_pred))
    sens.append(recall_score(y_test, y_pred))
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    specs.append(tn/(tn+fp))
    precs.append(precision_score(y_test, y_pred))
    
print('Accuracy:   ', np.mean(accs), 'se:', sem(accs))
print('Sensitivity:', np.mean(sens), 'se:', sem(sens))
print('Specificity:', np.mean(specs), 'se:', sem(specs))
print('Precision:  ', np.mean(precs), 'se:', sem(precs))

Accuracy:    0.8757948613515257 se: 0.0016729374973452693
Sensitivity: 0.8223203020693493 se: 0.0038455515652337643
Specificity: 0.9088867352418838 se: 0.003874115648592772
Precision:   0.8486979429267368 se: 0.005058536178514889


# Simple RASAR

In [4]:
from sklearn.linear_model import LogisticRegression

def right_neighbor(neighbors, X_train, X_train_i):
    # IDX Neighbors
    idx_neigh_0 = pd.DataFrame(neighbors[1])[0].apply(lambda x: X_train_i.iloc[x].name)
    idx_neigh_1 = pd.DataFrame(neighbors[1])[1].apply(lambda x: X_train_i.iloc[x].name)
    
    idx_neigh = idx_neigh_0.copy()
    
    # dove l'indice del primo vicino risulta essere uguale a se stesso lo sostituisco con il secondo vicino
    idx_neigh[X_train.index == idx_neigh_0] = idx_neigh_1[X_train.index == idx_neigh_0].values
    
    # Distance from the Nearest Neighbor that is NOT itself
    dist_0 = pd.DataFrame(neighbors[0])[0]
    dist_1 = pd.DataFrame(neighbors[0])[1]
    
    distance = dist_0.copy()
    distance[X_train.index == idx_neigh_0] = dist_1[X_train.index == idx_neigh_0].values
    
    return idx_neigh, distance

In [5]:
accs = []
sens = []
specs = []
precs = []

kf = KFold(n_splits=5, shuffle=True, random_state = 5645)

for train_index, test_index in kf.split(X):
    
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    minmax = MinMaxScaler()
    minmax.fit(X_train[numerical])

    new_train = X_train.copy()
    new_train.loc[:, numerical] = minmax.transform(X_train[numerical])

    new_test = X_test.copy()
    new_test.loc[:, numerical] = minmax.transform(X_test[numerical])
    
    X_train0 = X_train[y_train == 0].copy()
    X_train1 = X_train[y_train == 1].copy()

    ## DF Train

    knn0 = KNeighborsClassifier(n_neighbors = 2)
    knn0.fit(X_train0, y_train[y_train == 0])

    neigh_train0 = knn0.kneighbors(X_train, return_distance = True)
    idx_neigh0, dist0 = right_neighbor(neigh_train0, X_train, X_train0)

    knn1 = KNeighborsClassifier(n_neighbors = 2)
    knn1.fit(X_train1, y_train[y_train == 1])

    neigh_train1 = knn1.kneighbors(X_train, return_distance = True)
    idx_neigh1, dist1 = right_neighbor(neigh_train1, X_train, X_train1)

    df_rasar_train = pd.DataFrame({'idx_train': X_train.index.values,
                          'label_train': y_train,
                          'idx_neigh0': idx_neigh0.values,
                          'dist_neigh0': dist0,       
                          'idx_neigh1': idx_neigh1.values,
                          'dist_neigh1': dist1})

    # DF test
    knn0 = KNeighborsClassifier(n_neighbors = 1)
    knn0.fit(X_train0, y_train[y_train == 0])
    neigh_test0 = knn0.kneighbors(X_test, return_distance = True)
    idx_neigh_test0 = pd.DataFrame(neigh_test0[1])[0].apply(lambda x: X_train0.iloc[x].name)

    knn1 = KNeighborsClassifier(n_neighbors = 1)
    knn1.fit(X_train1, y_train[y_train == 1])
    neigh_test1 = knn1.kneighbors(X_test, return_distance = True)
    idx_neigh_test1 = pd.DataFrame(neigh_test1[1])[0].apply(lambda x: X_train1.iloc[x].name)

    df_rasar_test = pd.DataFrame({'idx_test': X_test.index.values,
                  'label_test': y_test,
                  'idx_neigh0': idx_neigh_test0.values,
                  'dist_neigh0': neigh_test0[0].ravel(),
                  'idx_neigh1': idx_neigh_test1.values,
                  'dist_neigh1': neigh_test1[0].ravel()})


    lrc = LogisticRegression()
    lrc.fit(df_rasar_train[['dist_neigh0','dist_neigh1']], y_train)
    y_pred = lrc.predict(df_rasar_test[['dist_neigh0','dist_neigh1']])
    accs.append(accuracy_score(y_test, y_pred))
    sens.append(recall_score(y_test, y_pred))
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    specs.append(tn/(tn+fp))
    precs.append(precision_score(y_test, y_pred))
    
print('Accuracy:   ', np.mean(accs), 'se:', sem(accs))
print('Sensitivity:', np.mean(sens), 'se:', sem(sens))
print('Specificity:', np.mean(specs), 'se:', sem(specs))
print('Precision:  ', np.mean(precs), 'se:', sem(precs))

Accuracy:    0.6985793090254958 se: 0.001083627388248233
Sensitivity: 0.9662125884843853 se: 0.0019552932698066795
Specificity: 0.5323428798141054 se: 0.004218068777537254
